In [1]:
import ltcode as lt
from ltcode import LOG
import dnaIO as dna
fileName = "data/hat-768x512.png"
outFile = "data/hat-768x512-out.png"

hashInit = dna.compute_file_hash(fileName)
LOG.Basic("INIT", "File Hash: %s" % hashInit)

[18:36:54] [INIT] File Hash: 5e9f48bd613eacfb6252c77ae7306a73


In [2]:
# 载入并切分文件
fileBlocks, fileBlocksN, dropQuantity, fileSize = lt.load_files(fileName, 256, 2)

# 生成喷泉码
fileSymbols = lt.create_symbols(fileBlocks, dropQuantity)

[18:36:55] [INIT] Filesize: 502888 bytes
[18:36:55] [INIT] Blocks: 983 -> 1966 * 512 bytes
[18:36:55] [ENCODE] Start Encode
[18:36:55] [LT-ENCODE] Generating graph...
[18:36:55] [LT-ENCODE] Ready for encoding.
[18:36:55] [LT-ENCODE] Correctly dropped 1966 symbols (packet size=65536)
[18:36:55] [ENCODE] End Encode


In [3]:
# 生成拼接比特流
bitsArr = dna.convert_bytes_form_symbols(fileSymbols, fileSize, blocksN=fileBlocksN)

[18:36:56] [ASSEM-ENCODE] Convert Symbol -> Bytes Start
[18:36:56] [ASSEM] Convert Symbol -> Bytes Finish


In [4]:
# 生成DNAArr
dnaArr = dna.convert_dnaArr_from_bitArr(bitsArr)

[18:36:57] [DNA-ENCODE] Convert Bytes -> DNA Start
[18:37:02] [DNA-ENCODE] Convert Bytes -> DNA Finish


In [5]:
# 进行模拟损毁
dnaArrOut = dna.destory_DNAs(dnaArr, 0.05, 10)

[18:37:24] [DESTORY] Destroy DNA 1966 --PCR-> 19660 --Destory(0.050000)-> 18736


In [6]:
# 使用DNA进行恢复为比特流
bitsArrOut = dna.convert_bitArr_from_DNAs(dnaArrOut)

[18:37:26] [DNA-DECODE] Convert DNA -> Bytes Start
[18:37:34] [DNA-DECODE] Convert DNA -> Bytes Finish


In [7]:
# 从比特流恢复到喷泉码
symbolsOut, fileSizeOut, fileBlocksNOut = dna.convert_symbols_from_bitArray(bitsArrOut)

[18:37:36] [ASSEM-DECODE] Convert Bytes -> Symbols Start
[18:37:36] [ASSEM-DECODE] Parse Legal Symbol: 18736 / 18736 FileSize: 502888 BlockN: 983
[18:37:36] [ASSEM-DECODE] Convert Bytes -> Symbols FINISH


In [8]:
# 使用喷泉码进行恢复
recoveredBlocks, recoverdN = lt.decode_from_symbols(symbolsOut, fileBlocksNOut)

# 导出恢复的文件
with open(outFile, "wb") as file:
    lt.blocks_write(recoveredBlocks, file, fileSizeOut)

hashOut = dna.compute_file_hash(outFile)
LOG.Basic("FINISH", "OutFile Hash: %s" % hashOut)
LOG.Basic("FINISH", "File Convert %s" % ("Success" if hashOut == hashInit else "Failed"))

[18:37:38] [DECODE] Start Decode
[18:37:38] [LT-DECODE] Graph built back. Ready for decoding.
[18:37:39] [LT-DECODE] ----- Solved Blocks 983/983 --
[18:37:39] [DECODE] End Decode
[18:37:39] [FINISH] OutFile Hash: 5e9f48bd613eacfb6252c77ae7306a73
[18:37:39] [FINISH] File Convert Success
